# 네이버 지도 맛집 검색 결과 mydb에 navermap 테이블에 저장

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup as bs 
import time  
import pandas as pd  
import os  
from urllib.request import urlretrieve
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

# ChromeDriverManager를 이용해서 크롬드라이버를 설치
serv = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=serv)

driver.get('https://map.naver.com/p?c=15.00,0,0,0,dh')

# 검색어 설정
search_key = '이수역 맛집'

# 검색어 입력 후 검색
try:
    search = driver.find_element(By.CSS_SELECTOR, '.input_search')
    time.sleep(1)  # 페이지 로드 시간 고려
    search.send_keys(search_key)
    search.send_keys(Keys.ENTER)
    time.sleep(3)  # 검색 결과가 나올 시간을 줄여서 3초로 설정
except NoSuchElementException:
    print("검색 입력란을 찾을 수 없습니다.")
    driver.quit()

# iframe으로 전환
try:
    driver.switch_to.frame('searchIframe')
    time.sleep(1)  # 프레임 전환 후 약간의 딜레이 추가
except NoSuchElementException:
    print("searchIframe을 찾을 수 없습니다.")
    driver.quit()

# 데이터 수집을 위한 리스트 초기화
tit_list = [] 
addr_list = [] 
num_list = [] 
tm_list = [] 

# 상호명 수집
titles = driver.find_elements(By.CSS_SELECTOR, 'span.place_bluelink.TYaxT')

for i, title in enumerate(titles):
    try:
        tit_list.append(title.text.strip())  # 상호명 리스트에 추가
        print(f"상호명: {tit_list[i]}")  # 상호명 출력하여 확인

        title.click()  # 상호명 클릭
        time.sleep(2)  # 클릭 후 페이지가 로드될 시간을 줄여서 2초로 설정
        
        # iframe 이동 후 데이터 수집은 클릭 이후에 한 번만 발생
        driver.switch_to.default_content()
        time.sleep(1)  # 프레임 복귀 후 약간의 딜레이
        driver.switch_to.frame('entryIframe')
        time.sleep(1)  # 데이터가 로드될 시간을 줄여서 1초로 설정
        
        # 주소 가져오기
        try:
            addr = driver.find_element(By.CSS_SELECTOR, '.LDgIH')
            addr_list.append(addr.text.strip())
            print(f"주소: {addr_list[i]}")  # 주소 출력하여 확인
        except NoSuchElementException:
            addr_list.append("주소 정보 없음")
            print("주소 정보 없음")  # 주소 정보 없음 출력

        # 전화번호 가져오기
        try:
            num = driver.find_element(By.CSS_SELECTOR, '.xlx7Q')
            num_list.append(num.text.strip())
            print(f"전화번호: {num_list[i]}")  # 전화번호 출력하여 확인
        except NoSuchElementException:
            num_list.append("전화번호 정보 없음")
            print("전화번호 정보 없음")  # 전화번호 정보 없음 출력

        # 영업시간 데이터 수집
        try:
            driver.find_element(By.CSS_SELECTOR, '.w9QyJ.vI8SM ._UCia').click()  # 영업시간 더보기 클릭
            time.sleep(1)  # 더보기 클릭 후 1초로 딜레이 감소
            tm_elements = driver.find_elements(By.CSS_SELECTOR, '.w9QyJ')
            tml = tm_elements[0].text.strip()
            tm_list.append(tml)
            # 요일별 영업시간 출력 시도 그러나 뒤의 필요없는 값까지 불러와짐
            # tml = [tm.text.replace('\n접기', '').replace('\n', '').strip() for tm in tm_elements[1:]]
            # tm_list.append(tml)
            print(f"영업시간: {tm_list[i]}")  # 영업시간 출력하여 확인
        except (NoSuchElementException, ElementClickInterceptedException):
            tm_list.append(["영업시간 정보 없음"])
            print("영업시간 정보 없음")  # 영업시간 정보 없음 출력

        # 원래 프레임으로 복귀
        driver.switch_to.default_content()
        time.sleep(1)  # 프레임 복귀 후 약간의 딜레이
        driver.switch_to.frame('searchIframe')
        time.sleep(1)  # 프레임 전환 후 약간의 딜레이

    except ElementClickInterceptedException:
        print(f"{i + 1}번째 상호 클릭 중 오류 발생")
        # 오류 발생 시에도 리스트에 빈 값 추가하여 리스트 길이를 맞춤
        addr_list.append("주소 정보 없음")
        num_list.append("전화번호 정보 없음")
        tm_list.append(["영업시간 정보 없음"])
        continue

# 크롬 브라우저 종료
driver.quit()


# 데이터프레임으로 변환 및 저장
dic = {'상호명': tit_list, '주소': addr_list, '전화번호': num_list, '영업시간': tm_list}

for d in dic:
    print(d)
print('끝')

In [1]:
#네이버 지도 맛집 검색 결과
#mydb에 navermap 테이블에 저장하기

#제출용

#필요한 모듈 불러오기~
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd

# 검색어 입력
# keyword = input('검색어를 입력해주세요: \n')
keyword = '이수역 맛집 \n'

serv = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=serv)

# 네이버 지도 페이지 열기
driver.get(f"https://map.naver.com/p?c=15.00,0,0,3,dh")
time.sleep(1)

search = driver.find_element(By.CSS_SELECTOR, '.input_search')
search.send_keys(f'{keyword}')
search.send_keys(Keys.ENTER)
time.sleep(2)

# html 안의 html로 들어가기
driver.switch_to.frame("searchIframe") 
time.sleep(2)

# 클릭할 가게 이름 찾아오기
title = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")

import pymysql as psq

conn = psq.connect(host= 'localhost', user= 'root', password= 'mysql904#', db= 'mydb', charset= 'utf8')
cur = conn.cursor()
cur.execute('''
    create table navermap(
    num int auto_increment primary key, 
    name varchar(40) not null, 
    major varchar(30) not null, 
    address varchar(100) not null, 
    tel varchar(20) not null, 
    open varchar(20) not null, 
    opentime varchar(50) not null, 
    star varchar(20) not null, 
    visitrev varchar(40) not null,
    blogrev varchar(40) not null, 
    link varchar(200) not null
)''')

j = 1 #카운터용
# DATA =[]
print(f"\n총 {len(title)}개를 검색하였습니다\n")
for i in range(len(title)): #매장개수 만큼 클릭하게 설정
    title[i].find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").click()  #매장 클릭
    time.sleep(2)
    
    driver.switch_to.default_content()  # 먼저 기존의 iframe에서 빠져나오기
    driver.switch_to.frame("entryIframe") #새로운 html 들어가기
    time.sleep(2)

   # navermap 테이블, 필드명(num, name, major, address, tel, open, opentime, visitrev, blogrev, star, link)
   # 크롤링 해오기
    이름 = driver.find_element(By.CSS_SELECTOR, ".GHAhO").text.strip()
    업종 = driver.find_element(By.CSS_SELECTOR, ".lnJFt").text.strip()
    주소 = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text.strip()
    번호 = driver.find_element(By.CSS_SELECTOR, ".xlx7Q").text.strip()
    영업 = driver.find_element(By.CSS_SELECTOR, ".y6tNq em").text.strip()
    시간 = driver.find_element(By.CSS_SELECTOR, ".y6tNq time").text.strip()
    방문 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/visitor']").text.strip()
    리뷰 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/ugc']").text.strip()
    try: # 별점이 없는 경우 
        별점 = driver.find_element(By.CSS_SELECTOR, ".PXMot.LXIwF").text.strip().replace("\n", "").replace("별점", "★")
    except: # 링크 없는 경우 처리하기 위함
        별점 = "Null"
    try: # 링크가 없는 경우가 있음 
        링크 = driver.find_element(By.CSS_SELECTOR, ".place_bluelink.CHmqa").get_attribute("href")
    except: # 링크 없는 경우 처리하기 위함
        링크 = "Null"

    time.sleep(2)
    if 별점 == 링크 == 'Null':
        cur.execute(f"insert into navermap(name, major, address, tel, open, opentime, star, visitrev, blogrev, link) values('{이름}', '{업종}', '{주소}', '{번호}', '{영업}', '{시간}', {별점}, '{방문}', '{리뷰}',{링크})")
    elif 별점 == 'Null':
        cur.execute(f"insert into navermap(name, major, address, tel, open, opentime, star, visitrev, blogrev, link) values('{이름}', '{업종}', '{주소}', '{번호}', '{영업}', '{시간}', {별점}, '{방문}', '{리뷰}','{링크}')")
    elif 링크 == 'Null':
        cur.execute(f"insert into navermap(name, major, address, tel, open, opentime, star, visitrev, blogrev, link) values('{이름}', '{업종}', '{주소}', '{번호}', '{영업}', '{시간}', '{별점}', '{방문}', '{리뷰}',{링크})")
    else:
        cur.execute(f"insert into navermap(name, major, address, tel, open, opentime, star, visitrev, blogrev, link) values('{이름}', '{업종}', '{주소}', '{번호}', '{영업}', '{시간}', '{별점}', '{방문}', '{리뷰}','{링크}')")
    print(f"{j}. 매장명: {이름}\n업종: {업종}\n주소: {주소}\n연락처: {번호}\n영업현황: {영업}\n영업시간: {시간}\n별점: {별점} / {방문} / {리뷰}\n링크: {링크}")
    print("="*40)
    # DATA.append([이름,업종,주소,번호,영업,시간,별점,방문,리뷰,링크])
    j += 1    
    driver.switch_to.default_content()  # 기존 html 나가기
    driver.switch_to.frame("searchIframe") 
    time.sleep(2)
conn.commit()
conn.close()

# 드라이버 종료
driver.quit()


총 10개를 검색하였습니다



IntegrityError: (1048, "Column 'star' cannot be null")

In [1]:
# 필요한 모듈 불러오기
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pymysql as psq

# 검색어 입력
keyword = '이수역 맛집'

# 크롬 드라이버 설정
serv = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=serv)

# 네이버 지도 페이지 열기
driver.get("https://map.naver.com/p?c=15.00,0,0,3,dh")
time.sleep(1)

# 검색어 입력
search = driver.find_element(By.CSS_SELECTOR, '.input_search')
search.send_keys(keyword)
search.send_keys(Keys.ENTER)
time.sleep(2)

# iframe 전환
driver.switch_to.frame("searchIframe")
time.sleep(2)

# 클릭할 가게 이름 찾아오기
title = driver.find_elements(By.CSS_SELECTOR, ".UEzoS.rTjJo")

# MySQL 연결
conn = psq.connect(host='localhost', user='root', password='mysql904#', db='mydb', charset='utf8')
cur = conn.cursor()

# 테이블 생성
cur.execute('''
    CREATE TABLE IF NOT EXISTS navermap(
        num INT AUTO_INCREMENT PRIMARY KEY, 
        name VARCHAR(40) NOT NULL, 
        major VARCHAR(30) NOT NULL, 
        address VARCHAR(100) NOT NULL, 
        tel VARCHAR(20) NOT NULL, 
        open VARCHAR(20) NOT NULL, 
        opentime VARCHAR(50) NOT NULL, 
        star VARCHAR(20), 
        visitrev VARCHAR(40),
        blogrev VARCHAR(40), 
        link VARCHAR(200)
    )
''')

j = 1 # 카운터용
print(f"\n총 {len(title)}개를 검색하였습니다\n")
for i in range(len(title)):  # 매장 개수만큼 클릭하게 설정
    title[i].find_element(By.CSS_SELECTOR, ".place_bluelink.TYaxT").click()  # 매장 클릭
    time.sleep(2)
    
    # iframe 전환
    driver.switch_to.default_content()
    driver.switch_to.frame("entryIframe")
    time.sleep(2)

    # 데이터 크롤링
    이름 = driver.find_element(By.CSS_SELECTOR, ".GHAhO").text.strip()
    업종 = driver.find_element(By.CSS_SELECTOR, ".lnJFt").text.strip()
    주소 = driver.find_element(By.CSS_SELECTOR, ".LDgIH").text.strip()
    번호 = driver.find_element(By.CSS_SELECTOR, ".xlx7Q").text.strip()
    영업 = driver.find_element(By.CSS_SELECTOR, ".y6tNq em").text.strip()
    시간 = driver.find_element(By.CSS_SELECTOR, ".y6tNq time").text.strip()
    방문 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/visitor']").text.strip()
    리뷰 = driver.find_element(By.CSS_SELECTOR, ".PXMot a[href*='/review/ugc']").text.strip()
    try:  # 별점이 없는 경우 
        별점 = driver.find_element(By.CSS_SELECTOR, ".PXMot.LXIwF").text.strip().replace("\n", "").replace("별점", "★")
    except:
        별점 = None
    try:  # 링크가 없는 경우가 있음 
        링크 = driver.find_element(By.CSS_SELECTOR, ".place_bluelink.CHmqa").get_attribute("href")
    except:
        링크 = None

    # 데이터베이스에 저장
    cur.execute(f"""
        INSERT INTO navermap(name, major, address, tel, open, opentime, star, visitrev, blogrev, link) 
        VALUES (
            '{이름}', 
            '{업종}', 
            '{주소}', 
            '{번호}', 
            '{영업}', 
            '{시간}', 
            {'NULL' if 별점 is None else f"'{별점}'"}, 
            '{방문}', 
            '{리뷰}', 
            {'NULL' if 링크 is None else f"'{링크}'"}
        )
    """)

    print(f"{j}. 매장명: {이름}\n업종: {업종}\n주소: {주소}\n연락처: {번호}\n영업현황: {영업}\n영업시간: {시간}\n별점: {별점} / {방문} / {리뷰}\n링크: {링크}")
    print("="*40)

    j += 1    
    driver.switch_to.default_content()  # 기존 html 나가기
    driver.switch_to.frame("searchIframe")
    time.sleep(2)

# 커밋 및 연결 종료
conn.commit()
conn.close()

# 드라이버 종료
driver.quit()


총 10개를 검색하였습니다

1. 매장명: 정선콩국수 소소정
업종: 한식
주소: 서울 서초구 동광로 68 1층 정선콩국수 소소정
연락처: 0507-1433-8667
영업현황: 곧 브레이크타임
영업시간: 15:00에 브레이크타임
별점: None / 방문자 리뷰 616 / 블로그 리뷰 207
링크: https://blog.naver.com/jin508815
2. 매장명: 지즐미육회관 사당점
업종: 소고기구이
주소: 서울 서초구 방배천로2길 39 1층
연락처: 0507-1492-3356
영업현황: 영업 중
영업시간: 21:00에 라스트오더
별점: ★4.73 / 방문자 리뷰 618 / 블로그 리뷰 152
링크: https://blog.naver.com/hanafc3334
3. 매장명: 애플하우스
업종: 떡볶이
주소: 서울 동작구 동작대로27다길 29 2층
연락처: 02-595-1629
영업현황: 영업 중
영업시간: 20:30에 라스트오더
별점: ★4.34 / 방문자 리뷰 12,513 / 블로그 리뷰 8,571
링크: None
4. 매장명: 외양간
업종: 육류,고기요리
주소: 서울 동작구 동작대로11길 16 1층
연락처: 0507-1484-8223
영업현황: 곧 영업 시작
영업시간: 15:00에 영업 시작
별점: ★4.69 / 방문자 리뷰 595 / 블로그 리뷰 702
링크: https://www.instagram.com/cowhome_sadang/
5. 매장명: 목구멍 이수점
업종: 육류,고기요리
주소: 서울 동작구 동작대로27길 16-4
연락처: 0507-1417-1959
영업현황: 영업 전
영업시간: 16:00에 영업 시작
별점: None / 방문자 리뷰 288 / 블로그 리뷰 2,313
링크: https://www.moggumung.com/
6. 매장명: 팔각도 이수역점
업종: 닭갈비
주소: 서울 동작구 동작대로27다길 12 1층
연락처: 0507-1340-0420
영업현황: 영업 중
영업시간: 22:10에 라스트오더
별점: ★4.72 / 방문자 리뷰 4,69